# Classic_Recommendation_Models

In [1]:
# Load the data
import pandas as pd

data_path = 'C:/Users/taek5/Desktop/TG/RecommenderSystems/data/book_crossing/'

books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)
users = pd.read_csv(data_path+'BX-Users-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False,encoding="latin-1", index_col = 0)
ratings = pd.read_csv(data_path+'BX-Book-Ratings-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)

C:\Users\taek5\AppData\Local\Temp\ipykernel_12824\3587479969.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)
C:\Users\taek5\AppData\Local\Temp\ipykernel_12824\3587479969.py:6: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1", index_col = 0)
C:\Users\taek5\AppData\Local\Temp\ipykernel_12824\3587479969.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(data_path+'BX-Books-comma-sep.csv', sep=',', error_bad_lines=False, warn_bad_lines=False, encoding="latin-

In [2]:
# Filter books with at least min_book_ratings ratings
min_book_ratings = 10
book_ratings_counts = ratings['Book-Title'].value_counts()
filtered_books = book_ratings_counts[book_ratings_counts >= min_book_ratings].index
ratings = ratings[ratings['Book-Title'].isin(filtered_books)]

In [3]:
# Filter users with at least min_user_ratings books
min_user_ratings = 5
user_rating_counts = ratings['User-ID'].value_counts()
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index
ratings = ratings[ratings['User-ID'].isin(filtered_users)]
ratings

,User-ID,Book-Rating,Book-Title
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel
5,23768,0,Flesh Tones: A Novel
...,...,...,...
1031125,276688,0,Coyote Waits
1031126,276688,0,Sacred Clowns
1031128,276688,0,Mrs. Pollifax and the Second Thief
1031131,276688,0,Mostly Harmless


In [4]:
# Create the user-item matrix
user_item_matrix = ratings.pivot_table(index = 'User-ID', columns = 'Book-Title', values = 'Book-Rating').fillna(0)

In [13]:
# Split user indices into train and test sets
import numpy as np
from sklearn.model_selection import train_test_split

user_indices = np.arange(user_item_matrix.shape[0])
train_user_indices, test_user_indices = train_test_split(user_indices, test_size = 0.001, random_state = 42)

In [15]:
def random_recommendation(ratings, n = 10):
    unique_books = ratings['Book-Title'].unique()
    random_books = np.random.choice(unique_books, size = n, replace = False)
    return random_books

random_books = random_recommendation(ratings, n = 10)
print('Random Score-Based Recommendations:')
for i,book in enumerate(random_books, 1):
    print(f"{i}. {book}")

Random Score-Based Recommendations:
1. Das Versprechen. Requiem auf den Kriminalroman.
2. The Delaney Woman
3. In God We Trust: All Others Pay Cash
4. Secrets of the Night
5. Venus Envy
6. The Bishop and the Three Kings: A Blackie Ryan Mystery (Blackie Ryan Novels)
7. The Discworld Mapp
8. The Red Scream
9. Q
10. Hullabaloo in the Guava Orchard


In [16]:
def popularity_recommendation(ratings, n = 10):
    popular_books = ratings.groupby('Book-Title')['Book-Title'].count().sort_values(ascending = False).head(n).index
    return popular_books

popular_books = popularity_recommendation(ratings, n = 10)
print("\nPopularity-Based Recommendations:")
for i, book in enumerate(popular_books, 1):
    print(f"{i}. {book}")


Popularity-Based Recommendations:
1. Wild Animus
2. The Lovely Bones: A Novel
3. The Da Vinci Code
4. The Nanny Diaries: A Novel
5. Bridget Jones's Diary
6. A Painted House
7. The Secret Life of Bees
8. Divine Secrets of the Ya-Ya Sisterhood: A Novel
9. Angels &amp; Demons
10. Life of Pi


## Association_Rules

In [72]:
# Load the data from the UCI Machine Learning Repository
import pandas as pd

data_path = 'C:/Users/taek5/Desktop/TG/RecommenderSystems/data/online_retail/'
online_retail_data = pd.read_excel(data_path + 'Online Retail.xlsx')

In [73]:
# Data preprocessing
online_retail_data['Description'] = online_retail_data['Description'].str.strip()
online_retail_data = online_retail_data.dropna(axis = 0, subset = ['InvoiceNo'])
online_retail_data['InvoiceNo'] = online_retail_data['InvoiceNo'].astype('str')
online_retail_data = online_retail_data[~online_retail_data['InvoiceNo'].str.contains('C')] # 취소

In [57]:
# Convert transaction data into a basker foramt
basket = (online_retail_data[online_retail_data['Country'] == 'United Kingdom']
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Convert the quantities into 0/1 (0: not in the basket, 1: in the basket)
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_sets = basket.applymap(encode_units)

# Use the Apriori algorithm to find frequent itemsets
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(basket_sets, min_support = 0.03, use_colnames = True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 1)

C:\Users\taek5\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [63]:
# Filter rules by a minimum lift and confidence
filtered_rules = rules[(rules['lift'] >= 3.0) & (rules['confidence'] >= 0.5)]

In [65]:
# Print the association rules
filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.030160,0.642694,12.900183
1,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.030160,0.605376,12.900183
2,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.030910,0.617773,16.403939
3,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.030910,0.820768,16.403939
4,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.037553,0.750535,14.639752
5,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037553,0.732497,14.639752
6,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.030535,0.626374,6.033290
8,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.042053,0.677308,6.523895
10,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.035196,0.579876,5.585425
13,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.037392,0.617699,5.949737


## Contents Filtering

In [7]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define the matrix
matrix = np.array([[1, 1, 1, 0],
                   [1, 0, 1, 1],
                   [1, 0, 0, 1],
                   [0, 1, 1, 0]])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(matrix)

# Print the similarity matrix
print(similarity_matrix)

[[1.         0.66666667 0.40824829 0.81649658]
 [0.66666667 1.         0.81649658 0.40824829]
 [0.40824829 0.81649658 1.         0.        ]
 [0.81649658 0.40824829 0.         1.        ]]


### Bag of Words 예제

In [106]:
import pandas as pd
from collections import Counter

# Sample sentences
documents = [
    "I love watching movies",
    "I enjoy reading books",
    "I love reading and watching movies"
]

# Preprocess sentences: lower case and tokenize
tokenized_documents = [doc.lower().split() for doc in documents]
print(tokenized_documents)

print('\n')
# Calculate word frequencies for each document
word_frequencies = [Counter(doc) for doc in tokenized_documents]
print(word_frequencies)

print('\n')
# Create a DataFrame to display the results
bow_df = pd.DataFrame(word_frequencies, index = ['Document 1', 'Document 2', 'Document 3']).fillna(0)
print("Bag of Words representation:")
display(bow_df)

[['i', 'love', 'watching', 'movies'], ['i', 'enjoy', 'reading', 'books'], ['i', 'love', 'reading', 'and', 'watching', 'movies']]


[Counter({'i': 1, 'love': 1, 'watching': 1, 'movies': 1}), Counter({'i': 1, 'enjoy': 1, 'reading': 1, 'books': 1}), Counter({'i': 1, 'love': 1, 'reading': 1, 'and': 1, 'watching': 1, 'movies': 1})]


Bag of Words representation:


,i,love,watching,movies,enjoy,reading,books,and
Document 1,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0
Document 2,1,0.0,0.0,0.0,1.0,1.0,1.0,0.0
Document 3,1,1.0,1.0,1.0,0.0,1.0,0.0,1.0


### TF-IDF 예제

In [91]:
from collections import Counter
import math

# Sample sentences
documents = [
    "I love watching movies",
    "I enjoy reading books",
    "I love reading and watching movies"
]

# Preprocess sentences: lower case and tokenize
tokenized_documents = [doc.lower().split() for doc in documents]
print("tokenized_documents:")
print(tokenized_documents)
print('\n')

# Calculate term frequency
def term_frequency(doc):
    term_count = Counter(doc)
    
    total_terms = len(doc)
    return {term: count / total_terms for term, count in term_count.items()}

tf_values = [term_frequency(doc) for doc in tokenized_documents]
print(tf_values)
print('\n')

tf_df = pd.DataFrame(tf_values, index = ["Document 1", "Document 2", "Document 3"]).fillna(0)
print("Term Frequencies DataFrame:")
display(tf_df)
print('\n')

# Calculate inverse document frequency
def inverse_document_frequency(docs):
    total_docs = len(docs)
    
    unique_terms = set(term for doc in docs for term in doc)
    
    term_doc_counts = {term: sum(1 for doc in docs if term in doc) for term in unique_terms}
    
    return {term: math.log(total_docs / count) for term, count in term_doc_counts.items()}

idf_values = inverse_document_frequency(tokenized_documents)
idf_df = pd.DataFrame([idf_values], index = ['IDF'])
print("Inverse Document Frequencies DataFrame:")
display(idf_df)
print('\n')

# Calculate TF-IDF
def tf_idf(tf, idf):
    return {term: tf_val * idf[term] for term, tf_val in tf.items()}

tf_idf_values = [tf_idf(tf, idf_values) for tf in tf_values]
print("TF-IDF Values:")
display(tf_idf_values)

tokenized_documents:
[['i', 'love', 'watching', 'movies'], ['i', 'enjoy', 'reading', 'books'], ['i', 'love', 'reading', 'and', 'watching', 'movies']]


[{'i': 0.25, 'love': 0.25, 'watching': 0.25, 'movies': 0.25}, {'i': 0.25, 'enjoy': 0.25, 'reading': 0.25, 'books': 0.25}, {'i': 0.16666666666666666, 'love': 0.16666666666666666, 'reading': 0.16666666666666666, 'and': 0.16666666666666666, 'watching': 0.16666666666666666, 'movies': 0.16666666666666666}]


Term Frequencies DataFrame:


,i,love,watching,movies,enjoy,reading,books,and
Document 1,0.250000,0.250000,0.250000,0.250000,0.00,0.000000,0.00,0.000000
Document 2,0.250000,0.000000,0.000000,0.000000,0.25,0.250000,0.25,0.000000
Document 3,0.166667,0.166667,0.166667,0.166667,0.00,0.166667,0.00,0.166667




Inverse Document Frequencies DataFrame:


,books,enjoy,reading,watching,movies,i,love,and
IDF,1.098612,1.098612,0.405465,0.405465,0.405465,0.0,0.405465,1.098612




TF-IDF Values:


[{'i': 0.0,
  'love': 0.1013662770270411,
  'watching': 0.1013662770270411,
  'movies': 0.1013662770270411},
 {'i': 0.0,
  'enjoy': 0.27465307216702745,
  'reading': 0.1013662770270411,
  'books': 0.27465307216702745},
 {'i': 0.0,
  'love': 0.06757751801802739,
  'reading': 0.06757751801802739,
  'and': 0.1831020481113516,
  'watching': 0.06757751801802739,
  'movies': 0.06757751801802739}]